In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor as MOR
from sklearn.metrics import r2_score
from joblib import dump

In [3]:
df = pd.read_excel(r"E:\Projects\UrbanHub\files\NewDataset.xlsx")
df

,ID,Green space ratio,Rotation,Main street,Sub street,Bldg Footprint,Type,Bldg Centroids x,Bldg Centroids y,Lengths,...,Aspect Ratio,Annual Solar Hours,Roof radiation - Cold,Roof radiation - Hot,SVF,Ave. UTCI - Cold,Ave. UTCI - Hot,Ave. Percent of Shaded area,Total EUI - Cold,Total EUI - Hot
0,B0,0.2,0,20,10,0.6,1,20.000000,30.666667,22.000000,...,0.9,448638,1158.21,3071.83,82.19,-0.47,37.07,9,8.653,8.695
1,B1,0.2,0,20,10,0.6,1,10.000000,30.666667,22.000000,...,0.6,299175,1034.74,2696.77,82.19,-0.47,37.07,9,8.653,8.695
2,B2,0.2,0,20,10,0.6,1,20.000000,77.333333,22.000000,...,0.6,264846,1042.35,2689.40,78.23,-0.47,37.07,9,8.653,8.695
3,B3,0.2,0,20,10,0.6,1,10.000000,77.333333,22.000000,...,0.9,431566,1153.40,3056.60,78.23,-0.47,37.07,9,8.653,8.695
4,B4,0.2,0,20,10,0.6,1,20.000000,124.000000,22.000000,...,0.6,265691,1060.30,2746.95,76.76,-0.47,37.07,9,8.653,8.695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,P0,0.2,90,25,5,0.6,0,-13.333333,103.750000,18.333333,...,0.0,0,0.00,0.00,84.17,0.01,36.59,16,8.779,10.197
15892,P1,0.2,90,25,5,0.6,0,33.333333,103.750000,18.333333,...,0.0,0,0.00,0.00,70.53,0.01,36.59,16,8.779,10.197
15893,P2,0.2,90,25,5,0.6,0,10.000000,58.750000,18.333333,...,0.0,0,0.00,0.00,62.54,0.01,36.59,16,8.779,10.197
15894,P3,0.2,90,25,5,0.6,0,33.333333,36.250000,18.333333,...,0.0,0,0.00,0.00,73.99,0.01,36.59,16,8.779,10.197


In [4]:
X = df[["Rotation","Bldg Centroids x", "Bldg Centroids y","Bldg Footprint", "Type", "Lengths", "Widths", "Stories"]]
y = df[[
        "Density", "Visibility", "Cooling - Cold", "Heating - Cold", "Lighting - Cold", "Hot water - Cold",
        "Gas - Cold", "Cooling - Hot", "Heating - Hot", "Lighting - Hot", "Hot water - Hot", "Gas - Hot", "Compactness 1",
        "Shape Factor", "Aspect Ratio", "Annual Solar Hours", "Roof radiation - Cold", "Roof radiation - Hot"
        ]]

In [5]:
X

,Rotation,Bldg Centroids x,Bldg Centroids y,Bldg Footprint,Type,Lengths,Widths,Stories
0,0,20.000000,30.666667,0.6,1,22.000000,10.000000,3
1,0,10.000000,30.666667,0.6,1,22.000000,10.000000,2
2,0,20.000000,77.333333,0.6,1,22.000000,10.000000,2
3,0,10.000000,77.333333,0.6,1,22.000000,10.000000,3
4,0,20.000000,124.000000,0.6,1,22.000000,10.000000,2
...,...,...,...,...,...,...,...,...
15891,90,-13.333333,103.750000,0.6,0,18.333333,18.333333,0
15892,90,33.333333,103.750000,0.6,0,18.333333,18.333333,0
15893,90,10.000000,58.750000,0.6,0,18.333333,18.333333,0
15894,90,33.333333,36.250000,0.6,0,18.333333,18.333333,0


In [6]:
y

,Density,Visibility,Cooling - Cold,Heating - Cold,Lighting - Cold,Hot water - Cold,Gas - Cold,Cooling - Hot,Heating - Hot,Lighting - Hot,Hot water - Hot,Gas - Hot,Compactness 1,Shape Factor,Aspect Ratio,Annual Solar Hours,Roof radiation - Cold,Roof radiation - Hot
0,2.5,0.00,2.77778,608.33382,130.55566,4827.78164,4844.44832,2588.89096,0,130.55566,3100.00248,3116.66916,3.62,0.40,0.9,448638,1158.21,3071.83
1,2.5,0.00,0.00000,600.00048,86.11118,3236.11370,3247.22482,1575.00126,0,86.11118,2083.33500,2094.44612,2.75,0.46,0.6,299175,1034.74,2696.77
2,2.5,0.00,0.00000,594.44492,86.11118,3236.11370,3247.22482,1522.22344,0,86.11118,2083.33500,2097.22390,2.75,0.46,0.6,264846,1042.35,2689.40
3,2.5,0.00,0.00000,611.11160,130.55566,4833.33720,4850.00388,2544.44648,0,130.55566,3102.78026,3119.44694,3.62,0.40,0.9,431566,1153.40,3056.60
4,2.5,0.00,0.00000,600.00048,86.11118,3236.11370,3247.22482,1572.22348,0,86.11118,2083.33500,2094.44612,2.75,0.46,0.6,265691,1060.30,2746.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,6.5,3.29,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0.00,0.00,0.0,0,0.00,0.00
15892,6.5,5.12,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0.00,0.00,0.0,0,0.00,0.00
15893,6.5,5.01,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0.00,0.00,0.0,0,0.00,0.00
15894,6.5,4.44,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,0.00,0.00,0.0,0,0.00,0.00


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
model = MOR(XGBRegressor(n_estimators=300,learning_rate=0.1,max_depth=6))
model.fit(X, y)
[...]

[Ellipsis]

In [34]:
yp_train = model.predict(X_train)
yp_test = model.predict(X_test)

In [93]:
r_pr = [[45,79.648232,111.719300,0.6,1,22,10,7]]
pr_df = pd.DataFrame(r_pr, columns=model.feature_names_in_)
pr_res_df = pd.DataFrame(model.predict(pr_df), columns=y.columns).iloc[0,:]
results_pr = {'Real': y_test.iloc[0,:], 'Predicted': pr_res_df}
results_pr_df = pd.DataFrame(results_pr)
print("Predicted values are very close to the real values!")
results_pr_df

Predicted values are very close to the real values!


,Real,Predicted
Density,6.50000,5.274093e+00
Visibility,0.00000,-2.142124e-03
Cooling - Cold,530.55598,4.865220e+02
Heating - Cold,413.88922,6.017351e+02
Lighting - Cold,302.77802,3.029097e+02
Hot water - Cold,11200.00896,1.120565e+04
Gas - Cold,11238.89788,1.123390e+04
Cooling - Hot,10441.67502,9.536077e+03
Heating - Hot,0.00000,9.389304e-15
Lighting - Hot,302.77802,3.029186e+02


In [35]:
r2_train=r2_score(y_train, yp_train, multioutput='raw_values')
r2_test=r2_score(y_test, yp_test, multioutput='raw_values')

In [38]:
results = {'Target': y.columns , 'Test Score': r2_test, 'Train Score': r2_train}
results_df = pd.DataFrame(results)
results_df

,Target,Test Score,Train Score
0,Density,0.471680,0.475789
1,Visibility,0.937495,0.936255
2,Cooling - Cold,0.922597,0.918303
3,Heating - Cold,0.879631,0.884747
4,Lighting - Cold,0.997802,0.998306
5,Hot water - Cold,0.997816,0.998312
6,Gas - Cold,0.997752,0.998321
7,Cooling - Hot,0.994505,0.993622
8,Heating - Hot,0.000000,0.000000
9,Lighting - Hot,0.999580,0.999587


In [44]:
dump(model, 'XGB_model.joblib')

['XGB_model.joblib']